In [24]:
from agential.cog.functional.critic import _prompt_agent, _prompt_critic
from agential.cog.prompts.critic import (
    CRITIC_INSTRUCTION_HOTPOTQA, 
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC
)
from langchain.prompts import PromptTemplate
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.tools.google_serper.tool import GoogleSerperRun 

search = GoogleSerperRun(api_wrapper=GoogleSerperAPIWrapper())

In [2]:
from langchain_community.chat_models.openai import ChatOpenAI

import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI()

question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

answer = _prompt_agent(
    llm=llm, 
    keys={
        "question": question, 
        "examples": HOTPOTQA_FEWSHOT_EXAMPLES_COT
    }, 
    prompt_template=CRITIC_INSTRUCTION_HOTPOTQA
)

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


<PROMPT IN =======================================================================>
Q: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
A: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft. So the answer is: 1,800 to 7,000 ft.

Q: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
A: Let's think step by step. Milhouse was named after U.S. president Richard Nixon. So the answer is: Richard Nixon.

Q: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
A: Let's think step by step. Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture. So the answer is: The Saimaa Gesture.

Q: What profes

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


<PROMPT OUT =======================================================================>
Let's break it down step by step:
1. The person was once considered the best kickboxer in the world.
2. They have been involved in controversies related to unsportsmanlike conduct in the sport.
3. They have also been involved in crimes of violence outside of the ring.
Based on these clues, the person in question is likely Badr Hari, a Dutch-Moroccan kickboxer known for his skills in the ring but also for his controversial behavior and legal issues. So the answer is: Badr Hari.
<PROMPT OUT =======================================================================>


In [3]:
print(answer)

Let's break it down step by step:
1. The person was once considered the best kickboxer in the world.
2. They have been involved in controversies related to unsportsmanlike conduct in the sport.
3. They have also been involved in crimes of violence outside of the ring.
Based on these clues, the person in question is likely Badr Hari, a Dutch-Moroccan kickboxer known for his skills in the ring but also for his controversial behavior and legal issues. So the answer is: Badr Hari.


In [31]:
critic_prompt = PromptTemplate.from_template(CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA).format(
    **{
        "question": question,
        "examples": HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
        "answer": answer, 
        "critique": ""
    }
)

In [3]:
from guidance import models, select
llm = models.OpenAI("gpt-3.5-turbo-instruct")

In [4]:
llm + f'Do you want a joke or a poem? A ' + select(['joke', 'poem'])
